In [0]:
#get the file name from the adf
fileName = dbutils.widgets.get('fileName')
fileNameWithoutExt = fileName.split('.')[0]
print(fileNameWithoutExt)

Product


In [0]:
import pyspark.sql.functions as F
#from datetime import datetme as dt

#Just change all the values here based on the resource name you have created in your environemnt and workspace.

sqlDbName = 'ap-morgan-db'
sqlTableName = 'dbo.FileDetailsFormat'
dbUserName = 'connected'
passwordKey = 'sql-db-pass'
stgAccountSASTokenKey = 'sas-token-for-strg'
landingFileName =fileName #'Product'  #dbutils.widgets.get('Product')
databricksScopeName ='ap-morgan-scope'
dbServer = 'connected-vehicle'
dbServerPortNumber ='1433'
storageContainer ='input'
storageAccount='apmorganraw'
landingMountPoint ='/mnt'


In [0]:
if not any(mount.mountPoint == landingMountPoint for mount in dbutils.fs.mounts()):
    dbutils.fs.mount( source = 'wasbs://{}@{}.blob.core.windows.net'.format(storageContainer, storageAccount), mount_point= landingMountPoint, extra_configs ={'fs.azure.sas.{}.{}.blob.core.windows.net'.format(storageContainer,storageAccount):dbutils.secrets.get(scope = databricksScopeName, key= stgAccountSASTokenKey)})
    print('Mounted the storage account successfully')
else:
    print('Storage account already mounted')

Storage account already mounted


In [0]:
mounted_files = dbutils.fs.ls(landingMountPoint)
display(mounted_files)

path,name,size,modificationTime
dbfs:/mnt/landing/,landing/,0,0
dbfs:/mnt/rejected/,rejected/,0,0
dbfs:/mnt/staging/,staging/,0,0


In [0]:
#connect to Azure SQL DB
dbPassword = dbutils.secrets.get(scope = databricksScopeName, key= passwordKey)
serverurl = 'jdbc:sqlserver://{}.database.windows.net:{};database={};user={};'.format(dbServer, dbServerPortNumber,sqlDbName,dbUserName)
connectionProperties = {
    'password':dbPassword,
    'driver':'com.microsoft.sqlserver.jdbc.SQLServerDriver'
}
df = spark.read.jdbc(url = serverurl, table = sqlTableName, properties= connectionProperties)
display(df)


FileNo,FileName,ColumnName,ColumnDateFormat,ColumnIsNull,ModifiedDate
1,Product,StartDate,MM-dd-yyyy,true,2012-06-18T22:34:09Z
1,Product,EndDate,MM/dd/yyyy,true,2012-06-18T22:34:09Z
1,Product,CreateDate,MM/dd/yyyy,true,2012-06-18T22:34:09Z
1,Product,ModifiedDate,MM/dd/yyyy,true,2012-06-18T22:34:09Z
2,ProductDescription,ModifiedDate,MM/dd/yyyy,true,2012-06-18T22:34:09Z
2,ProductDescription,StartDate,MM/dd/yyyy,true,2012-06-18T22:34:09Z
2,ProductDescription,EndDate,MM/dd/yyyy,true,2012-06-18T22:34:09Z
3,CustomerDetail,CreateDate,MM/dd/yyyy,true,2012-06-18T22:34:09Z
3,CustomerDetail,ActiveDate,MM/dd/yyyy,true,2012-06-18T22:34:09Z


In [0]:
df1 = spark.read.csv('/mnt/landing/'+fileName, inferSchema=True, header=True)
display(df1)

# Rule
errorFlag=False
errorMessage = ''
totalcount = df1.count()
print(totalcount)
distinctCount = df1.distinct().count()
print(distinctCount)
if distinctCount !=totalcount:
    errorFlag = True
    errorMessage = 'Duplication Found. Rule 1 Failed'
print(errorMessage)

ProductId,Price,guid,StartDate,EndDate,CreateDate,ModifiedDate
NO6034318860278,67 822,14A53EFA-97A9-D852-39F9-E1A366AED2E8,2021-07-27,04/22/2022,12-07-2019,06/16/2020
LU744388398764786734,41 965,07C6BA82-8664-4446-0D52-963BE523C5C7,2022-01-08,03/26/2022,01-01-2020,12-01-2020
IL622786216445251297137,79 352,A4B97E53-2B68-8271-B4C2-F598D1525D92,2021-06-21,04/13/2022,10/23/2019,09/26/2020
HU73530772784633198693907654,93 661,670A2B65-76A4-4B70-B13B-16240274C539,2021-07-29,04-09-2022,02/20/2020,10/19/2020
MT93HZCQ15847141393282243268486,51 231,A2C6ADE3-ABB1-8EC1-5414-8A895C5CA2A4,2021-01-20,03/28/2022,12/22/2019,07/20/2020
ES5071462808326222586356,62 696,E30FC4A9-A2A2-AAB0-58A2-C1382B647D46,2021-09-30,02/26/2022,01-03-2020,06/13/2020
BG44MNMM33864028879242,37 332,E762D650-18D0-3860-A67F-21F0CCE483CD,2021-04-28,06/23/2022,04-12-2020,12/15/2020
EE034518283401745714,85 982,E591D646-4190-4860-8974-D0CC4DE965C6,2021-04-28,12-10-2022,09/25/2019,06/29/2020
IS171916556906574422538815,1 040,C5BAC88D-81A3-3B45-DD0B-6C89EEF896BA,2021-04-27,12-01-2022,09-07-2019,11-06-2020
IE08OFCR35332787214285,82 632,70746AD7-A219-0023-87BF-66F35FC9D77C,2021-01-13,09-01-2022,09/20/2019,11-11-2020


501
500
Duplication Found. Rule 1 Failed


In [0]:
# Rule 2
df2 = df.filter(df.FileName==fileNameWithoutExt).select('ColumnName','ColumnDateFormat' )
rows = df2.collect()
for r in rows:
    colName = r[0]
    colFormat =r[1]
    print(colName, colFormat)
    #display(df1.filter(F.to_date(colName, colFormat).isNull() ==True))
    formatCount =df1.filter(F.to_date(colName, colFormat).isNotNull() ==True).count()
    if formatCount != totalcount:
        errorFlag = True
        errorMessage = errorMessage +' DateFormate is incorrect for {} '.format(colName)
    else:
        print('All rows are good for ', colName)
print(errorMessage)

StartDate MM-dd-yyyy
All rows are good for  StartDate
EndDate MM/dd/yyyy
CreateDate MM/dd/yyyy
ModifiedDate MM/dd/yyyy
Duplication Found. Rule 1 Failed DateFormate is incorrect for EndDate  DateFormate is incorrect for CreateDate  DateFormate is incorrect for ModifiedDate 


In [0]:
if errorFlag:
    dbutils.fs.mv('/mnt/landing/'+fileName,'/mnt/rejected/'+fileName )
    dbutils.notebook.exit('{"errorFlag": "true", "errorMessage":"'+errorMessage +'"}')
else:
    dbutils.fs.mv('/mnt/landing/'+fileName,'/mnt/staging/'+fileName )
    dbutils.notebook.exit('{"errorFlag": "false", "errorMessage":"No error"}')